# Source Phase analysis for $\left| \phi^+ \rangle \right.$ - Pair 4

In [3]:
import glob
import itertools
import pandas as pd
import numpy as np
import scipy as sp
from scipy import linalg,stats
from scipy.optimize import least_squares,minimize
#from numpy import random, linalg
from numpy.random import default_rng
import matplotlib.pyplot as plt
#import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm,ticker,colors,rc,font_manager
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
%matplotlib inline
import os
try:
    fm = font_manager.json_load(os.path.expanduser("~/.cache/matplotlib/fontlist-v310.json"))
except:
    fm = font_manager.json_load(os.path.expanduser("~/.cache/matplotlib/fontlist-v300.json"))
fm.findfont("serif", rebuild_if_missing=False)
rc('font',**{'family':'sans-serif','sans-serif':['Computer Modern Sans serif']})
## for Palatino and other serif fonts use:
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)
plt.rcParams.update({'font.size': 12})

## Some experimental parameters

In [41]:
# number of qubits
n_qbits = 2
# laser repetition rate
R = 8E7
# number of measurements
measurements = 2
# seconds per line of datafile (measurement time)
meas_time = 1
# motor angles
#mot_ang = [6,7,8]
#mot_ang = [-10,-3,2,6,9,11,13,14,15,16]
# detector lists [A1 B1 A2 B2]
addresses_sin = {'h1':1,'v1':2,'h2':3,'v2':4}
#correct for datafiles
datafile_list = False
#for key in addresses_sin:
#    addresses_sin[key] +=1
#print(addresses_sin)
#folder with measurments
fold = './'

# function to create list of coincidences of n_qubits
def coinc_str_first(n,coinc_list=['h','v']):
    outs = ['h','v']
    coinc_tmp = []
    for i in coinc_list:
        for k in outs:
            coinc_tmp.append(i+k)
    if n==1:
        return outs
    elif n==2:
        return coinc_tmp
    else:
        return coinc_str_first(n-1,coinc_tmp)
    
def coinc_str(n):
    tmp_str = coinc_str_first(n)
    tmp2 = tmp_str.copy()
    for i,entry in enumerate(tmp2):
        tmp3 = ''
        for j,det in enumerate(entry):
            tmp3 += det+str(j+1)
        tmp_str[i] = tmp3
    return tmp_str

# angles for each setting in order [QWP,HWP]
#settings = [[-15,-30] for i in range(measurements)]
# actual settings
settings = [[0,0],[0,22.5]]

repeat = 1000 #number of monte-carlo generations
#m_per_turn = 300 #number of measurements per turn
mot_err = 0.2 #motor error in degrees

# correct for accidentals? 'theory' or 'no' (or 'exp' for 2 photons)
acc_corr = 'theory'

## QI definitions

In [34]:
#convert deg to rad
def theta(x):
    return x*np.pi/180

# Lambda-Half wave plate
def HWP(x):
    return np.array([[np.cos(2*theta(x)),np.sin(2*theta(x))],[np.sin(2*theta(x)),-np.cos(2*theta(x))]])

# QWP
def QWP(x):
    return np.array([[np.cos(theta(x))**2+1j*np.sin(theta(x))**2,(1-1j)*np.sin(theta(x))*np.cos(theta(x))],
            [(1-1j)*np.sin(theta(x))*np.cos(theta(x)),np.sin(theta(x))**2+1j*np.cos(theta(x))**2]])

#pauli matrices
s0 = np.eye(2)
sx = np.array([[0, 1],[ 1, 0]])
sy = np.array([[0, -1j],[1j, 0]])
sz = np.array([[1, 0],[0, -1]])

#basis states
h = np.array([1, 0])
v = np.array([0, 1])

# Generate measurement operator for n qubits as tensor product of the same measurement sigma
def meas_multi(n,sigma):
    if n==1:
        return sigma
    else:
        return np.kron(meas_multi(n-1,sigma),sigma)

# GHZ^n state
def GHZ(nqub=6):
    ghz = np.zeros((2**nqub,1))
    ghz[0],ghz[-1] = 1/np.sqrt(2),1/np.sqrt(2)
    return ghz

# visibility
def visib(setting, data):
    if setting==0:
        #vis = (data[0]+data[-1])/sum(data)
        #actual sigma_Z visibility
        vis = (data[0]+data[-1]-sum(data[1:-1]))/sum(data)
    elif setting in range(1,len(data)):
        sett_str = coinc_str(n_qbits)
        data_plus = []
        data_minus = []
        for i,sett in enumerate(sett_str):
            if (sett.count('h')%2 == 0):
                data_plus.append(i)
            else:
                data_minus.append(i)
        num = sum([data[i] for i in data_plus]) - sum([data[i] for i in data_minus])
        vis = num/sum(data)
    else:
        print('Wrong setting number!')
    return vis

def visiX(data):
    num = sum([data[i] for i in [0,3,5,6,9,10,12,15]]) - sum([data[i] for i in [1,2,4,7,8,11,13,14]])
    vis = num/sum(data)
    return vis

#file_lenght
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

#file_len(fold+'1_datafile.dat')

### Define addresses to import, with labels.
Also define accidentals

In [32]:
def Address(add_dict,datafile_list):
    addresses = {}
    dets = []
    for i in range(int(len(add_dict)/2)):
        dets.append(dict(itertools.islice(add_dict.items(), 2*i, 2*i+2)))

    if len(add_dict)//2 == 6:
        # 6-photon addresses
        for comb6 in map(dict,itertools.product(*[dets[i].items() for i in range(len(dets))])):
            addresses[''.join([key for key in comb6])] = sum([2**(list(comb6.values())[i]-1) for i in range(len(comb6))])
            # 5-photon addresses
            for comb5 in map(dict,itertools.combinations(comb6.items(),5)):
                addresses[''.join([key for key in comb5])] = sum([2**(list(comb5.values())[i]-1) for i in range(len(comb5))])
                # 4-photon addresses
                for comb4 in map(dict,itertools.combinations(comb5.items(),4)):
                    addresses[''.join([key for key in comb4])] = sum([2**(list(comb4.values())[i]-1) for i in range(len(comb4))])
                    # 3-photon addresses
                    for comb3 in map(dict,itertools.combinations(comb4.items(),3)):
                        addresses[''.join([key for key in comb3])] = sum([2**(list(comb3.values())[i]-1) for i in range(len(comb3))])
                        # 2-photon addresses
                        for comb2 in map(dict,itertools.combinations(comb3.items(),2)):
                            addresses[''.join([key for key in comb2])] = sum([2**(list(comb2.values())[i]-1) for i in range(len(comb2))])
                            # 1-photon addresses
                            for comb1 in map(dict,itertools.combinations(comb2.items(),1)):
                                addresses[''.join([key for key in comb1])] = sum([2**(list(comb1.values())[i]-1) for i in range(len(comb1))])
        # list of 5-photon accidentals
        acc_list = []
        for comb in map(dict,itertools.product(*[dets[i].items() for i in range(len(dets))])):
            lst = list(map(dict,itertools.combinations(comb.items(),5)))
            tmp = []
            for el in comb:
                for item in lst:
                    if el not in item:
                        tmp.append([''.join([key for key in item]),el])
            acc_list.append(tmp) 
    
    elif len(add_dict)//2 == 4:
        # 4-photon addresses
        for comb4 in map(dict,itertools.product(*[dets[i].items() for i in range(len(dets))])):
            addresses[''.join([key for key in comb4])] = sum([2**(list(comb4.values())[i]-1) for i in range(len(comb4))])
            # 3-photon addresses
            for comb3 in map(dict,itertools.combinations(comb4.items(),3)):
                addresses[''.join([key for key in comb3])] = sum([2**(list(comb3.values())[i]-1) for i in range(len(comb3))])
                # 2-photon addresses
                for comb2 in map(dict,itertools.combinations(comb3.items(),2)):
                    addresses[''.join([key for key in comb2])] = sum([2**(list(comb2.values())[i]-1) for i in range(len(comb2))])
                    # 1-photon addresses
                    for comb1 in map(dict,itertools.combinations(comb2.items(),1)):
                        addresses[''.join([key for key in comb1])] = sum([2**(list(comb1.values())[i]-1) for i in range(len(comb1))])
        # list of 3-photon accidentals
        acc_list = []
        for comb in map(dict,itertools.product(*[dets[i].items() for i in range(len(dets))])):
            lst = list(map(dict,itertools.combinations(comb.items(),3)))
            tmp = []
            for el in comb:
                for item in lst:
                    if el not in item:
                        tmp.append([''.join([key for key in item]),el])
            acc_list.append(tmp) 
            
    elif len(add_dict)//2 == 2:
        # 2-photon addresses
        for comb2 in map(dict,itertools.product(*[dets[i].items() for i in range(len(dets))])):
            addresses[''.join([key for key in comb2])] = sum([2**(list(comb2.values())[i]-1) for i in range(len(comb2))])
            # 1-photon addresses
            for comb1 in map(dict,itertools.combinations(comb2.items(),1)):
                addresses[''.join([key for key in comb1])] = sum([2**(list(comb1.values())[i]-1) for i in range(len(comb1))])
        # list of 1-photon accidentals
        acc_list = []
        for comb in map(dict,itertools.product(*[dets[i].items() for i in range(len(dets))])):
            lst = list(map(dict,itertools.combinations(comb.items(),1)))
            tmp = []
            for el in comb:
                for item in lst:
                    if el not in item:
                        tmp.append([''.join([key for key in item]),el])
            acc_list.append(tmp) 
    
    # add 2-photon experimental accidentals
    for i,comb in enumerate(dets):
        addresses[''.join([key for key in comb])] = sum([2**(list(comb.values())[i]-1) for i in range(len(comb))])
        
    #sort dictionary
    addresses = {k: v for k, v in sorted(addresses.items(), key=lambda item: item[1])}
    # fill lists of columns to import and their labels
    # name colunms
    cols_to_imp = []
    cols_labels = []

    for el in addresses:
        cols_labels.append(el)
        cols_to_imp.append(addresses[el])

    # correct addresses if datafile comes from list
    if datafile_list:
        for key in addresses:
            addresses[key] +=1
        cols_to_imp_tmp = cols_to_imp.copy()
        for i,el in enumerate(cols_to_imp_tmp):
            cols_to_imp[i] +=1
    
    return [addresses,cols_to_imp,cols_labels,acc_list]

#Address(addresses_sin, datafile_list)

## Reduce and clean files

In [11]:
# Reduce datafiles
# what's the highest address needed?
def reduce_files(addresses_sin, datafile_lst=datafile_list, folder=fold):
    max_add = 1
    for det in addresses_sin:
        max_add += 2**(addresses_sin[det]-1)

    print(max_add)
    # find all data files
    all_dat_files = glob.glob(folder+'*.dat')
    dat_files = []
    for fil in all_dat_files:
        if 'red' not in fil:
            dat_files.append(fil)
    print(dat_files)

    # clean files
    # list of measurements?
    if datafile_lst:
        for file_num,fil in enumerate(dat_files):
            with open(fil) as in_f:
                # list to contain cleaned data
                cl_data = ''
                for line in in_f:
                    arry = [x for x in line.split()]
                    if arry[1]=='0':
                        for num in arry[:max_add]:
                            cl_data += num
                            cl_data += ' '
            #            cl_data += arry[max_add]
                        cl_data += '\n'
                with open(str(fil)[:-4]+'_red.dat', 'w') as o_f:
                    o_f.write(cl_data)
        
    elif not datafile_lst:
        for file_num,fil in enumerate(dat_files):
            with open(fil) as in_f:
                # list to contain cleaned data
                cl_data = ''
                for line in in_f:
                    arry = [x for x in line.split()]
                    if arry[0]=='0':
                        for num in arry[:max_add]:
                            cl_data += num
                            cl_data += ' '
            #            cl_data += arry[max_add]
                        cl_data += '\n'
                with open(str(fil)[:-4]+'_red.dat', 'w') as o_f:
                    o_f.write(cl_data)
                    
    else:
        print("Are all datafiles the same (list or not list)?")

reduce_files(addresses_sin)

16
['./2_da.dat', './1_hv.dat']


## Import data

In [35]:
def file_list(setting, folder=fold):
    # find all data files

    dat_files = glob.glob(folder+'*_red.dat')
    dat_files = sorted(dat_files, key=lambda item: int(item.split('_')[0][len(folder):])) #sort datafiles

    # find out min lenght
    
    m_per_turn = file_len(dat_files[setting])

    return dat_files[setting],m_per_turn

def data_import(setting, datafile_lst=datafile_list, folder=fold):
# find all data files

    dat_file, m_per_turn = file_list(setting,folder)
    
    #m_per_turn = 4000 # override m_per_turn
    skip_rows = 0 # skip initial rows
    m_per_turn = m_per_turn-skip_rows
    counts_sing_turn = [] #holds the real data
    
    adds = Address(addresses_sin,datafile_lst)
#   df = pd.read_csv(dat_file, sep=' ', header=None, names=Address(addresses_sin)[2], usecols=Address(addresses_sin)[1], nrows=min(meas_len), engine='python')
    df = pd.read_csv(dat_file, sep=' ', header=None, names=adds[2], usecols=adds[1], skiprows=skip_rows, nrows=m_per_turn, dtype={'user_id': int})
    setting_dict = {}
    # loop on turns
    for add in adds[0]:
        setting_dict[add] = df[add].sum()
    counts_sing_turn.append(setting_dict)

    print(setting,dat_file)
    
    return [counts_sing_turn,m_per_turn]

# generate monte-carlo data, 'repeat' amount of sets
def MC_data(real_data, reps):
    
    counts = [real_data.copy()] #will hold total counts for each address, for one setting, for each repetition
    
    if reps<2:
        return counts
    
    for turn in range(reps-1):
        new_data_round = [] #will hold new MC-generated data, for one settings
#        counter = 0

        new_data_dict = {} #will hold new MC-generated data, for one settings

        for el in real_data[0]:
            new_data_dict[el] = np.random.default_rng().normal(real_data[0][el],np.sqrt(real_data[0][el]))
        new_data_round.append(new_data_dict)
        counts.append(new_data_round)

    return counts

#print(len(MC_data(data_import(),reps=3)))

def exp_freq(setting, reps, meas_tm=meas_time, datafile_lst=datafile_list, folder=fold):
# now create vectors for experimental frequencies
    f_v = [] #n-photon counts

    data = data_import(setting, datafile_lst, folder)
    counts = MC_data(data[0],reps)
    print('Lines in datafile: {}'.format(data[1]))
    coincs = coinc_str(n_qbits)
    
    for i,turn in enumerate(counts):
        f_v_sing_turn = []
        for m_d in counts[i]:
            if acc_corr=='theory':
                acc_all = Address(addresses_sin, datafile_lst)[3]
                acc = []
                for row in acc_all:
                    tot = 0
                    for pair in row:
                        tot += m_d[pair[0]]*m_d[pair[1]]
                    acc.append(tot/(n_qbits*R*data[1]*meas_tm))
                norm = 0
                for i,comb in enumerate(coincs):
                    norm += max(m_d[comb]-acc[i],0)

                for i,comb in enumerate(coincs):
                    f_v_sing_turn.append(max((m_d[comb]-acc[i])/norm,0))
            elif acc_corr=='exp': #HARD-CODED: only works for 2 photons
                hh = m_d['h1h2']-(m_d['h1v1']+m_d['h2v2'])/2
                hv = m_d['h1v2']-(m_d['h1v1']+m_d['h2v2'])/2
                vh = m_d['v1h2']-(m_d['h1v1']+m_d['h2v2'])/2
                vv = m_d['v1v2']-(m_d['h1v1']+m_d['h2v2'])/2
                norm = hh + hv + vh + vv
                f_v_sing_turn.append(hh/norm)
                f_v_sing_turn.append(hv/norm)
                f_v_sing_turn.append(vh/norm)
                f_v_sing_turn.append(vv/norm)
            elif acc_corr=='no':
                norm = sum(m_d[ff] for ff in coincs)
                #print(norm)
                for comb in coincs:
                    f_v_sing_turn.append(m_d[comb]/norm)
            else: print('Error with understanding if we want to correct for accs or not')
        f_v.append(f_v_sing_turn)
    return f_v

#function to extract total counts and rate per second for a list of coincidences (e.g. 'h1v2v3h4h5h6')
def tot_rate(keys, setting, meas_tm=meas_time, datafile_lst=datafile_list, folder=fold):
    counts,m_per_turn = data_import(setting, datafile_lst,folder)
    
    counts_dict = counts[0]
    #total
    tot = 0
    for key in counts_dict:
        if ('1' in key) and ('2' in key):
            tot += counts_dict[key]
            
    print('Total 2-photon events: {}. Average rate: {:.2f} Hz'.format(tot,tot/m_per_turn/meas_time))
    p_coinc = (counts_dict['h1h2']+counts_dict['h1v2']+counts_dict['v1h2']+counts_dict['v1v2'])/(m_per_turn*meas_time)
                                                          
    print('Avg rate: {:.0f} Hz'.format(p_coinc))
    
    for key in keys:
        print(key)
        print('Total: ', counts_dict[key], '; Rate: ', counts_dict[key]/m_per_turn/meas_time)

    return tot,tot/m_per_turn/meas_time,p_coinc

#tot_rate([],0)

rates = [tot_rate([],i) for i in range(len(settings))]

print('Overall:')
print('2-photon events: {}. Average rate: {:.2f} Hz'.format(sum([rates[i][0] for i in range(len(rates))]),
                                                            sum([rates[i][1] for i in range(len(rates))])/len(rates)))
#print('Avg coincs: P1: {:.0f} Hz, P2: {:.0f} Hz, P3: {:.0f} Hz'.format(sum([rates[i][2] for i in range(len(rates))])/len(rates),
#                                                                      sum([rates[i][3] for i in range(len(rates))])/len(rates),
#                                                                      sum([rates[i][4] for i in range(len(rates))])/len(rates)))

0 ./1_hv_red.dat
Total 2-photon events: 13917336. Average rate: 100124.72 Hz
Avg rate: 100125 Hz
1 ./2_da_red.dat
Total 2-photon events: 12779485. Average rate: 98303.73 Hz
Avg rate: 98304 Hz
Overall:
2-photon events: 26696821. Average rate: 99214.23 Hz


## Functions for statistical and systematic error analysis

In [36]:
## Statistical error analysis
def visi_stat(setting, reps, datafile_lst=datafile_list, folder=fold):
    visi = [] # contains visibilities for all Poisson generated data-sets

    if setting in range(len(settings)):
        f_v = exp_freq(setting, reps, meas_time, datafile_lst, folder) # contains all data (first: real data; next: MC generated)
        for i in range(reps):
            visi.append(visib(setting,f_v[i]))
            
    else:
        print('Wrong setting number!')
        
    return visi

## Systematic error analysis
def visi_syst(setting, reps, mot_err):
    visi = [] # contains visibilities for all Poisson generated data-sets

    if setting in range(len(settings)):
        for i in range(reps):
            qwp = np.random.normal(settings[setting][0],mot_err)
            hwp = np.random.normal(settings[setting][1],mot_err)
            state = meas_multi(n_qbits,QWP(qwp)@HWP(hwp))@GHZ(n_qbits)
            #state = meas_multi(n_qbits,QWP(qwp)@HWP(hwp))@state1
            visi.append(visib(setting,np.real((state*state.conj()).T[0])))
            
    else:
        print('Wrong setting number!')
        
    return visi


## Results for the different measurement settings

In [44]:
visi = [] # will contain median visibility of all settings
visi_MC = [] # will contain median visibility of all settings
visi_err_stat = [] # will contain statistical errors, negative first row, positive second row
visi_err_syst = [] # will contain systematic errors, negative first row, positive second row

st_err_neg_tmp = []
st_err_pos_tmp = []
sy_err_neg_tmp = []
sy_err_pos_tmp = []
# repeat for each setting
for i in range(len(settings)):
    vis_tmp = visi_stat(i,repeat,datafile_list,fold)
    visi.append(vis_tmp[0])
    visi_MC.append(np.median(vis_tmp))
    #statistical error
    st_err_neg_tmp.append(np.median(vis_tmp)-np.percentile(vis_tmp,50-34.1))
    st_err_pos_tmp.append(np.percentile(vis_tmp,50+34.1)-np.median(vis_tmp))
    #systematic error
    vis_sys_tmp = visi_syst(i,repeat,mot_err)
    sy_err_neg_tmp.append(np.median(vis_sys_tmp)-np.percentile(vis_sys_tmp,50-34.1))
    sy_err_pos_tmp.append(np.percentile(vis_sys_tmp,50+34.1)-np.median(vis_sys_tmp))

visi_err_stat.append(st_err_neg_tmp)
visi_err_stat.append(st_err_pos_tmp)
visi_err_syst.append(sy_err_neg_tmp)
visi_err_syst.append(sy_err_pos_tmp)
    
print('Visibilities:')
print(visi)
print('Montecarlo sampling:')
print('Visibilities (MC) - {} reps:'.format(repeat))
print(visi_MC)
print('Statistical errors:')
print('-',visi_err_stat[0])
print('+',visi_err_stat[1])

print('Systematic errors:')
print('-',visi_err_syst[0])
print('+',visi_err_syst[1])

0 ./1_hv_red.dat
Lines in datafile: 139
1 ./2_da_red.dat
Lines in datafile: 130
Visibilities:
[0.9965549997929035, 0.9965558819948128]
Montecarlo sampling:
Visibilities (MC) - 1000 reps:
[0.996554524506782, 0.996555241728712]
Statistical errors:
- [0.00010755413585683549, 0.00011322067975305661]
+ [0.00010787531800271477, 0.0001135378464864889]
Systematic errors:
- [0.00013622091591858343, 0.00015426248298355816]
+ [3.74028424035e-05, 4.1162956170737175e-05]


## $\gamma_3^2$ - $\left| \phi^+ \rangle \right.$ - Fidelity results

In [45]:
#visi[0] = 0.912
fidelity = (1 + visi[0] + 2*visi[1])/(2*n_qbits)
#statistical
fid_stat_err_neg = np.sqrt(visi_err_stat[0][0]**2/16 + visi_err_stat[0][1]**2/4)
fid_stat_err_pos = np.sqrt(visi_err_stat[1][0]**2/16 + visi_err_stat[1][1]**2/4)
#systematic
fid_syst_err_neg = np.sqrt(visi_err_syst[0][0]**2/16 + visi_err_syst[0][1]**2/4)
fid_syst_err_pos = np.sqrt(visi_err_syst[1][0]**2/16 + visi_err_syst[1][1]**2/4)

print('Fidelity: ',fidelity)
print('Statistical errors:')
print('-',fid_stat_err_neg)
print('+',fid_stat_err_pos)

print('Systematic errors:')
print('-',fid_syst_err_neg)
print('+',fid_syst_err_pos)

Fidelity:  0.9974166909456323
Statistical errors:
- 6.267155526776453e-05
+ 6.284925147393608e-05
Systematic errors:
- 8.431480910800459e-05
+ 2.2606039655804895e-05


## Results

In [ ]:
visi = [0.9965549997929035, 0.9965558819948128]
visi_MC = [0.996554524506782, 0.996555241728712]
visi_err_stat = [[0.00010755413585683549, 0.00011322067975305661],[0.00010787531800271477, 0.0001135378464864889]]
visi_err_syst = [[0.00013622091591858343, 0.00015426248298355816],[3.74028424035e-05, 4.1162956170737175e-05]]